In [5]:
from Dataset.mnist import MNISTDataset

In [1]:
from lightning import Trainer
from torch.utils.data import DataLoader
from Model.mlp import MLP

In [2]:
import numpy as np

In [3]:
from torch.utils.data import Subset, DataLoader, ConcatDataset

In [9]:
dataset = MNISTDataset()

train_indices = np.arange(len(dataset.train_set))
data_size=2500
num_client=10
train_subsets_indices = [train_indices[i * data_size:(i + 1) * data_size] for i in range(num_client)]
train_subsets = [Subset(dataset.train_set, indices) for indices in train_subsets_indices]
 
for i in range(1):   
    train_loader = DataLoader(train_subsets[i], batch_size=128, shuffle=True, num_workers=0)
    model = MLP()
    local_trainer = Trainer(max_epochs=10, accelerator="gpu", devices=1, logger=False,
                                                            # callbacks = [MyCustomCheckpoint(save_dir=f"{model_directory}/Local_models/Round_{r}",
                                                            # idx=client.idx, rou=r, logger=model_logger)],
                                                            enable_checkpointing=False, enable_model_summary=False, enable_progress_bar=True)
                                            
    local_trainer.fit(model, train_loader)                                       
                                                                    

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
j:\TIA\.venv\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
You are using a CUDA device ('NVIDIA GeForce RTX 4070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 9: 100%|██████████| 20/20 [00:00<00:00, 119.79it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 20/20 [00:00<00:00, 119.79it/s]


In [8]:
from Dataset.svhn import SVHNdataset
from Model.svhnresnet18 import SVHNResNet18
dataset = SVHNdataset()

train_indices = np.arange(len(dataset.train_set))
data_size=2500
num_client=10
train_subsets_indices = [train_indices[i * data_size:(i + 1) * data_size] for i in range(num_client)]
train_subsets = [Subset(dataset.train_set, indices) for indices in train_subsets_indices]
 
for i in range(1):   
    train_loader = DataLoader(train_subsets[i], batch_size=128, shuffle=True, num_workers=0)
    model = SVHNResNet18()
    local_trainer = Trainer(max_epochs=10, accelerator="gpu", devices=1, logger=False,
                                                            # callbacks = [MyCustomCheckpoint(save_dir=f"{model_directory}/Local_models/Round_{r}",
                                                            # idx=client.idx, rou=r, logger=model_logger)],
                                                            enable_checkpointing=False, enable_model_summary=False, enable_progress_bar=True)
                                            
    local_trainer.fit(model, train_loader)    
test_indices = np.arange(len(dataset.test_set))
data_size=2500
num_client=10
test_subsets_indices = [test_indices[i * data_size:(i + 1) * data_size] for i in range(num_client)]
test_subsets = [Subset(dataset.test_set, indices) for indices in test_subsets_indices]
test_loader = DataLoader(test_subsets[0], batch_size=128, shuffle=True, num_workers=0)
local_trainer.test(model, test_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [7]:
from Dataset.pcam import PCAMdataset
from Model.pcamresnet18 import PCAMResNet18
dataset = PCAMdataset()

train_indices = np.arange(len(dataset.train_set))
data_size=2500
num_client=10
train_subsets_indices = [train_indices[i * data_size:(i + 1) * data_size] for i in range(num_client)]
train_subsets = [Subset(dataset.train_set, indices) for indices in train_subsets_indices]
 
for i in range(1):   
    train_loader = DataLoader(train_subsets[i], batch_size=128, shuffle=True, num_workers=0)
    model = PCAMResNet18()
    local_trainer = Trainer(max_epochs=10, accelerator="gpu", devices=1, logger=False,
                                                            # callbacks = [MyCustomCheckpoint(save_dir=f"{model_directory}/Local_models/Round_{r}",
                                                            # idx=client.idx, rou=r, logger=model_logger)],
                                                            enable_checkpointing=False, enable_model_summary=False, enable_progress_bar=True)
                                            
    local_trainer.fit(model, train_loader)    
test_indices = np.arange(len(dataset.test_set))
data_size=2500
num_client=10
test_subsets_indices = [test_indices[i * data_size:(i + 1) * data_size] for i in range(num_client)]
test_subsets = [Subset(dataset.test_set, indices) for indices in test_subsets_indices]
test_loader = DataLoader(test_subsets[0], batch_size=128, shuffle=True, num_workers=0)
local_trainer.test(model, test_loader)


j:\TIA\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
j:\TIA\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
j:\TIA\.venv\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
j:\TIA\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider incre

Epoch 0:   0%|          | 0/20 [00:00<?, ?it/s] 

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
j:\TIA\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:476: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
j:\TIA\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 20/20 [00:00<00:00, 24.91it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.7383655905723572
      test_f1_score         0.7413121461868286
        test_loss           0.8195502758026123
     test_precision         0.7775265574455261
       test_recall          0.7383655905723572
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.8195502758026123,
  'test_accuracy': 0.7383655905723572,
  'test_precision': 0.7775265574455261,
  'test_recall': 0.7383655905723572,
  'test_f1_score': 0.7413121461868286}]